In [ ]:
%pip install -qqqq folium openpyxl

In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import plotly.express as px
import plotly.io as pio

# Load the golf course data
df = pd.read_csv('data/Golf-Courses-USA.csv')

df[['name', 'citystate']] = df['name'].str.split('-', n=1, expand=True)
df[['description', 'street', 'city', 'statezip', 'phone']] = df['description'].str.split(',', n=4, expand=True)
df[['city', 'state']] = df['citystate'].str.split(',', n=1, expand=True)
df['zip'] = df['statezip'].str.extract(r'(\d{5}(?:-\d{4})?)')
df[['type', 'num_holes']] = df['description'].str.extract(r'\((.*?)\)\s+\((\d+)\s+Holes\)')

df['num_holes'] = df['num_holes'].fillna(9).astype(int)
df['state'] = df['state'].str.strip()

df = df[['longitude','latitude','name','type','num_holes','street','city','state','zip','phone']].reset_index(drop=True)

# Create a map centered on the USA
# The location is an approximate center of the contiguous United States.
map_center = [39.8283, -98.5795]
usa_map = folium.Map(location=map_center, zoom_start=4)

# Add a marker for each golf course
for index, row in df.iterrows():
    # Create a popup with the name of the golf course
    popup_text = f"<strong>{row['name']}</strong>"

    folium.Marker(
        location=[row['latitude'], row['longitude']],
        # popup=popup_text,
        popup=None,
        icon=folium.Icon(color='green', prefix = 'fa', icon='golf-ball-tee')
    ).add_to(usa_map)

# Save the map to an HTML file
output_file = 'output/golf_courses_map.html'
usa_map.save(output_file)

print(f"Interactive map saved to {output_file}")

In [ ]:
# Create map and add MarkerCluster
usa_map = folium.Map(location=map_center, zoom_start=4)
marker_cluster = MarkerCluster().add_to(usa_map)

# Add markers to the cluster
for _, row in df.iterrows():
    popup_text = f"<strong>{row['name']}</strong>"
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=popup_text,
        icon=folium.Icon(color='green', prefix='fa', icon='golf-ball-tee')
    ).add_to(marker_cluster)

# Save the map to an HTML file
output_file = 'output/golf_courses_map.html'
usa_map.save(output_file)

print(f"Interactive map saved to {output_file}")

In [ ]:
# The GCSAA and USGA surveys are industry-standard references, capturing real data from hundreds of U.S. golf facilities over multiple years (2005, 2013, 2020) 
gallons_9_hole = 12600000 # per year
gallons_18_hole = 21600000 # per year

# https://www.epa.gov/watersense/statistics-and-facts#:~:text=Each%20American%20uses%20an%20average,the%20United%20States%20in%202015).
gallons_per_person = 82 * 365

In [ ]:
water_usage = pd.DataFrame()
water_usage['num_holes'] = sorted(df['num_holes'].unique())

def compute_gallons(num):
    if num == 9:
        return gallons_9_hole
    elif num % 18 == 0:
        return gallons_18_hole * (num // 18)
    elif num % 9 == 0:
        num_18 = num // 18
        remainder = num % 18
        if remainder == 9:
            return gallons_18_hole * num_18 + gallons_9_hole
    return gallons_18_hole

water_usage['gallons_per_year'] = water_usage['num_holes'].apply(compute_gallons)
water_usage


In [ ]:

state_pop = pd.read_excel("data/NST-EST2024-POP.xlsx", skiprows=3)
state_pop.rename(columns={'Unnamed: 0': 'state_long',
                       2024 : 'population'}, inplace=True)
state_pop['state_long'] = state_pop['state_long'].str.replace('.', '', regex=False).str.strip()

state_pop = state_pop[['state_long','population']].dropna()

# US state name to abbreviation mapping
us_state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI',
    'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME',
    'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN',
    'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE',
    'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM',
    'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH',
    'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI',
    'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX',
    'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY',
    'Puerto Rico': 'PR'
}

# Add a new column with the abbreviation (NaN if not found)
state_pop['state'] = state_pop['state_long'].map(us_state_abbrev)

In [ ]:
summary = df.merge(water_usage, on='num_holes', how='left')
summary = summary.groupby(['state']).agg({'gallons_per_year': 'sum'}).reset_index()
summary = summary.merge(state_pop, on = 'state', how = 'left')
summary['gallons_per_person'] = gallons_per_person * summary['population']
summary['golfwater_div_pop'] = (summary['gallons_per_year'] / summary['population']).astype(int)
summary['population'] = summary['population'].astype(int)
summary

In [ ]:
fig = px.choropleth(
    summary,
    locations='state',
    locationmode='USA-states',
    color='golfwater_div_pop',
    color_continuous_scale=[(0, 'white'), (1, 'darkblue')],
    scope='usa',
    labels={'golfwater_div_pop': 'Golf Water Use per Person (gal/year)'},
    title='Golf Course Water Use per Person by State'
)
fig.update_layout(geo=dict(bgcolor='rgba(0,0,0,0)'))

pio.write_html(fig, file='output/heatmap-golfwater-perperson.html')